In [ ]:
# версии вебуи
newest_commit =  "5ef669de080814067961f28357256e8fe27544f4"
previous_commit = "c9c8485bc1e8720aba70f029d25cba1c4abf2b5c"
# элементы виджета
checkpoint = widgets.Dropdown(options=[('v1.6.0', 1), ('v1.5.2', 2)], value=1, description='версия A1111:').add_class("checkpoint")
opt = widgets.RadioButtons(options=['ничего', 'SDP', 'xformers'], value='xformers', description='Оптимизация:').add_class("opt")
extentions_mix = widgets.Checkbox(value=False, description='дополнения для слияние моделей').add_class("extentions_mix")
booru = widgets.Checkbox(value=False, description='DeepDanbooru').add_class("booru")
no_half_vae = widgets.Checkbox(value=True, description='--no-half-vae').add_class("no_half_vae")
starton = widgets.Checkbox(value=True, description='запустить сразу').add_class("starton")
eng = widgets.Checkbox(value=False, description='англоязычный интерфейс').add_class("eng")
ngrok_token = widgets.Text(value="", placeholder='тут можно указать свой токен ngrok').add_class("ngrok_token")
auth_token = widgets.HTML(value='''<a class="auth_token" title="получить свой токен Ngrok" href="https://dashboard.ngrok.com/get-started/your-authtoken" target="_blank">?</a>''')
additions = widgets.HBox([extentions_mix, booru]).add_class("additions")
webuiargs = widgets.HBox([no_half_vae, starton, eng, ngrok_token, auth_token]).add_class("args")
apply = widgets.Button(description='продолжить ❯').add_class("apply")
settings_first_header = HTML('<h2 id="setup_header">настройки</h2>')

# установка вебуи и основных зависимостей, правки
# функция правок в коде
def sed(file_path, line_to_replace, new_line):
  try:
    with open(file_path, 'r') as file: lines = file.readlines()
    with open(file_path, 'w') as file:
      for line in lines:
          if line_to_replace in line:
            line = line.replace(line_to_replace, new_line)
          file.write(line)
  except: pass
# функция удаления и замены строк в коде и текстах
def replacer(file_path, line_to_remove, new):
  try:
    with open(file_path) as r:
      text = r.read().replace(f"""{line_to_remove}""", f"""{new}""")
    with open(file_path, "w") as w: w.write(text)
  except: pass
# функция вставки абзаца с кодом в конец файла
def code2file_end(file_path, paragraph):
  try:
    import textwrap
    with open(file_path, 'a') as file: file.write('\n\n' + textwrap.dedent(paragraph))
  except: pass
# функция вставки абзаца с кодом в после определенной строки файла
def add_code(file_path, search_text, code):
  try:
    import textwrap
    with open(file_path, 'r') as file: lines = file.readlines()
    with open(file_path, 'w') as file:
      for line in lines:
        file.write(line)
        if search_text in line:
          code = code.lstrip('\n') if code.startswith('\n') else code
          file.write(code)
  except: pass
# функция установки
def A1111_webui_install():
  try:
    subprocess.run(["git", "clone", "https://github.com/AUTOMATIC1111/stable-diffusion-webui"])
    os.chdir('/content/stable-diffusion-webui')
    if checkpoint_value == "newest":
      subprocess.run(["git", "checkout", newest_commit], cwd="/content/stable-diffusion-webui")
    if checkpoint_value == "previous":
      subprocess.run(["git", "checkout", previous_commit], cwd="/content/stable-diffusion-webui")
      # замена строк из файла зависимостей, для предотвращения переустановки
      replacer("/content/stable-diffusion-webui/requirements_versions.txt", "gradio==3.32.0", "gradio==3.33.0")
      replacer("/content/stable-diffusion-webui/requirements.txt", "gradio==3.32.0", "gradio==3.33.0")
    # установка репозиториев для вебуи
    launch = "modules/launch_utils.py"
    try:
      with open(f'{launch}', 'r') as fp:
        lines = fp.readlines()
        for line in lines:
          if line.find('STABLE_DIFFUSION_COMMIT_HASH') != -1:
            stable_diffusion_commit_hash = line.split('"')[1]
          if line.find('STABLE_DIFFUSION_XL_COMMIT_HASH') != -1:
            stable_diffusion_xl_commit_hash = line.split('"')[1]
          if line.find('K_DIFFUSION_COMMIT_HASH') != -1:
            k_diffusion_commit_hash = line.split('"')[1]
          if line.find('CODEFORMER_COMMIT_HASH') != -1:
            codeformer_commit_hash = line.split('"')[1]
          if line.find('BLIP_COMMIT_HASH') != -1:
            blip_commit_hash = line.split('"')[1]
      reps = [
          f"git clone https://github.com/Stability-AI/stablediffusion.git /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai && cd /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai && git checkout {stable_diffusion_commit_hash}",
          f"git clone https://github.com/Stability-AI/generative-models.git /content/stable-diffusion-webui/repositories/generative-models && cd /content/stable-diffusion-webui/repositories/generative-models && git checkout {stable_diffusion_xl_commit_hash}",
          f"git clone https://github.com/crowsonkb/k-diffusion.git /content/stable-diffusion-webui/repositories/k-diffusion && cd /content/stable-diffusion-webui/repositories/k-diffusion && git checkout {k_diffusion_commit_hash}",
          f"git clone https://github.com/sczhou/CodeFormer.git /content/stable-diffusion-webui/repositories/CodeFormer && cd /content/stable-diffusion-webui/repositories/CodeFormer && git checkout {codeformer_commit_hash}",
          f"git clone https://github.com/salesforce/BLIP.git /content/stable-diffusion-webui/repositories/BLIP && cd /content/stable-diffusion-webui/repositories/BLIP && git checkout {blip_commit_hash}",
          f"git clone https://github.com/isl-org/MiDaS.git /content/stable-diffusion-webui/repositories/midas",
          "wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /usr/bin/cloudflared  && chmod +x /usr/bin/cloudflared",
          "pip install open-clip-torch==2.20.0",
          ]
      def reps_inst(rep):
        !$rep
      with ThreadPoolExecutor(max_workers=len(reps)) as executor:
        futures = [executor.submit(reps_inst, rep) for rep in reps]
        for future in as_completed(futures):
            future.result()
      openclip = "/root/.cache/huggingface/hub/models--openai--clip-vit-large-patch14/snapshots/8d052a0f05efbaefbc9e8786ba291cfdf93e5bff/"
      os.makedirs(openclip, exist_ok=True)
      gdshare_linker("interrogate/pytorch_model.bin", openclip)
    except:
      None
    !$wget $pt'nullModel.ckpt' -P /content/stable-diffusion-webui/models/Stable-diffusion/ > /dev/null 2>&1
    !$wget {deps}cache.tar
    !tar -xvf cache.tar -C /
    !rm cache.tar
    !pip install tomesd Pillow==9.5.0 > /dev/null 2>&1 # не помню для кого эта версия Pillow==8.4.0
    if checkpoint_value == "previous":
      !pip install gradio==3.33.0 > /dev/null 2>&1
    else:
      !pip install gradio==3.41.2 > /dev/null 2>&1
    os.environ["SAFETENSORS_FAST_GPU"]='1'
    os.chdir('/content/stable-diffusion-webui')
    if checkpoint_value == "previous":
      sed("/content/stable-diffusion-webui/webui.py", "ui.create_ui()", "ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)")
      replacer("/content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py", 'return get_obj_from_str(config["target"])(**config.get("params", dict()))', 'return get_obj_from_str(config["target"])(**config.get("params", dict())).cuda()')
      sed("/content/stable-diffusion-webui/modules/sd_models.py", "os.path.splitext(checkpoint_file)", "os.path.splitext(checkpoint_file); map_location='cuda'")
      sed("/content/stable-diffusion-webui/modules/extras.py", "map_location='cpu'", "map_location='cuda'")
      sed("/content/stable-diffusion-webui/modules/shared.py", 'cmd_opts.lowram else "cpu"', 'cmd_opts.lowram else "cuda"')
    clear()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции A1111_webui_install и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      error_message = str(traceback.format_exc())
      e = "в функции A1111_webui_install и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)


# скачивание моделей и дополнения ControlNet
def ControlNet_install():
  try:
    subprocess.run(["git", "clone", "--depth", "1", "https://github.com/Mikubill/sd-webui-controlnet"], cwd="/content/stable-diffusion-webui/extensions")
    replacer("/content/stable-diffusion-webui/extensions/sd-webui-controlnet/scripts/controlnet.py", '# "openpose_hand": openpose_hand,', '"openpose_hand": openpose_hand,')
    controlnets = [
        "pip install svglib > /dev/null 2>&1",
        "pip install mediapipe > /dev/null 2>&1",
        "pip install fvcore > /dev/null 2>&1",
        "git clone https://github.com/fkunn1326/openpose-editor /content/stable-diffusion-webui/extensions/openpose-editor",
        "git clone https://github.com/hnmr293/posex /content/stable-diffusion-webui/extensions/posex",
        "git clone https://github.com/nonnonstop/sd-webui-3d-open-pose-editor /content/stable-diffusion-webui/extensions/3d-open-pose-editor",
        "git clone https://github.com/jexom/sd-webui-depth-lib /content/stable-diffusion-webui/extensions/depth-lib",
      ]
    def controlnet_ext(controlnet):
        !$controlnet
    with ThreadPoolExecutor(max_workers=len(controlnets)) as executor:
        futures = [executor.submit(controlnet_ext, controlnet) for controlnet in controlnets]
        for future in as_completed(futures):
            future.result()
    controlnet_path = "/content/stable-diffusion-webui/extensions/sd-webui-controlnet/"
    controlnet_models_path = f"{controlnet_path}models"
    controlnet_annotators_path = f"{controlnet_path}annotator"
    !rm -rf $controlnet_models_path $controlnet_annotators_path > /dev/null 2>&1
    gdshare_linker("ControlNet/models", controlnet_path)
    gdshare_linker("ControlNet/annotator", controlnet_path)
    dmaps = "/content/stable-diffusion-webui/extensions/sd-webui-depth-lib/maps/"
    !rm -rf {dmaps}*
    subprocess.run(["wget", "-t", "10", "--content-disposition", "-O", "maps.7z", "https://raw.githubusercontent.com/PR0LAPSE/StableDiffusionWebUIColab/main/src/maps.7z"], cwd="/content/")
    !7z -bso0 -bd -mmt4 -slp -y x /content/maps.7z -o{dmaps}
    !rm /content/maps.7z
    clear()
except Exception as e:
  try:
    error_message = str(cgitb.text(sys.exc_info()))
    e = "в функции ControlNet_install и получил "+str(e)
    ErrorReport(e, error_message)
    print("Ошибка: ", e)
  except Exception as e:
    import traceback
    error_message = str(traceback.format_exc())
    e = "в функции ControlNet_install и получил "+str(e)
    ErrorReport(e, error_message)
    print("Ошибка: ", error_message)

# обязательные дополнения
#f"git clone https://github.com/wcde/custom-hires-fix-for-automatic1111 /content/stable-diffusion-webui/extensions/custom-hires-fix",
#f"git clone https://github.com/hnmr293/sd-webui-llul /content/stable-diffusion-webui/extensions/sd-webui-llul",
#f"git clone https://github.com/KohakuBlueleaf/a1111-sd-webui-locon /content/stable-diffusion-webui/extensions/a1111-sd-webui-locon",
#f"git clone https://github.com/richrobber2/canvas-zoom /content/stable-diffusion-webui/extensions/canvas-zoom",
#f"git clone https://github.com/PR0LAPSE/lora2prompt /content/stable-diffusion-webui/extensions-builtin/lora2prompt",
def extentions_builtin():
  try:
    #таймер, описания моделей и апскейлер, замена лиц, пропорции картинки, генератор промптов
    os.makedirs('/content/stable-diffusion-webui/static', exist_ok = True)
    builtins = [
      f"git clone https://github.com/PR0LAPSE/webui-fix /content/stable-diffusion-webui/extensions-builtin/fix-webui",
      f"git clone https://github.com/PR0LAPSE/photopea-a1111 /content/stable-diffusion-webui/extensions/photopea-a1111",
      f"git clone https://github.com/PR0LAPSE/prompt_translator /content/stable-diffusion-webui/extensions-builtin/prompt_translator",
      f"git clone https://github.com/PR0LAPSE/colab_sd_models /content/stable-diffusion-webui/extensions-builtin/colab_sd_models",
      f"git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111 /content/stable-diffusion-webui/extensions/ultimate-upscale-for-automatic1111",
      f"git clone https://github.com/adieyal/sd-dynamic-prompts /content/stable-diffusion-webui/extensions/sd-dynamic-prompts",
      f"git clone https://github.com/yownas/shift-attention /content/stable-diffusion-webui/extensions/shift-attention",
      f"git clone https://github.com/yownas/seed_travel /content/stable-diffusion-webui/extensions/seed_travel",
      f"git clone https://github.com/Filarius/video2video /content/stable-diffusion-webui/extensions/video2video",
      f"git clone https://github.com/Gourieff/sd-webui-roop-nsfw /content/stable-diffusion-webui/extensions/sd-webui-roop-nsfw",
      f"git clone https://github.com/pkuliyi2015/sd-webui-stablesr  /content/stable-diffusion-webui/extensions/sd-webui-stablesr",
      "pip install mediapipe dynamicprompts==0.29.0 > /dev/null 2>&1",
      f"git clone https://github.com/thomasasfk/sd-webui-aspect-ratio-helper /content/stable-diffusion-webui/extensions/sd-webui-aspect-ratio-helper",
      f"git clone https://github.com/Bing-su/adetailer.git /content/stable-diffusion-webui/extensions/adetailer",
    ]
    def ext_builtin_inst(builtin):
      !$builtin
    with ThreadPoolExecutor(max_workers=len(builtins)) as executor:
      futures = [executor.submit(ext_builtin_inst, builtin) for builtin in builtins]
      for future in as_completed(futures):
          future.result()
    if checkpoint_value == "previous":
      subprocess.run(["git", "checkout", "74502fe7c3cd819a176dbf40badca9972a6cd2f2"], cwd="/content/stable-diffusion-webui/extensions-builtin/fix-webui")
    !echo -n {launching} > /content/stable-diffusion-webui/static/launch.txt
    gdshare_linker("inswapper_128_fp16.onnx", "/content/stable-diffusion-webui/models/insightface")
    !pip install -r /content/stable-diffusion-webui/extensions/sd-webui-roop-nsfw/requirements.txt  > /dev/null 2>&1
    !pip install insightface==0.7.3 onnx==1.14.0 onnxruntime==1.15.0
    !$wget https://huggingface.co/utnah/deps/resolve/main/models--Bingsu--adetailer.zip -O /content/models--Bingsu--adetailer.zip
    !7z -bso0 -bd -mmt4 -slp -y x /content/models--Bingsu--adetailer.zip -o/root/.cache/huggingface/hub
    !rm /content/models--Bingsu--adetailer.zip
    !$wget $dvach'penis.pt' -P /content/stable-diffusion-webui/models/adetailer
    os.chdir('/content/stable-diffusion-webui')
    #скрипты
    scrurls = [
        "https://github.com/feffy380/prompt-morph/raw/master/prompt_morph.py",
        "https://github.com/Animator-Anon/Animator/raw/main/animation.py",
        "https://github.com/Animator-Anon/Animator/raw/main/animation_v6.py",
        "https://github.com/memes-forever/Stable-diffusion-webui-video/raw/main/videos.py",
        "https://github.com/dfaker/embedding-to-png-script/raw/main/embedding_to_png.py",
        "https://github.com/dfaker/stable-diffusion-webui-cv2-external-masking-script/raw/main/external_masking.py",
    ]
    def download_sdscripts(scrurl):
      !$dl $scrurl -P /content/stable-diffusion-webui/scripts
    with ThreadPoolExecutor(max_workers=len(scrurls)) as executor:
      futures = [executor.submit(download_sdscripts, scrurl) for scrurl in scrurls]
      for future in as_completed(futures):
          future.result()
    # дополнение clip-interrogator
    subprocess.run(["git", "clone", "--depth", "1", "https://github.com/pharmapsychotic/clip-interrogator-ext"], cwd="/content/stable-diffusion-webui/extensions")
    !pip install clip-interrogator==0.5.4 > /dev/null 2>&1
    for filename in os.listdir("/content/drive/MyDrive/colab_files/clip-interrogator"):
      gdshare_linker(f"clip-interrogator/{filename}", "/content/stable-diffusion-webui/models/clip-interrogator")
    clear()
  except Exception as e:
    try:
      e = "в функции extentions_builtin и получил "+str(e)
      error_message = str(cgitb.text(sys.exc_info()))
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      e = "в функции extentions_builtin и получил "+str(e)
      import traceback
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)

# скачивание пользовательских дополнений/скриптов display(own_extentions_links)
own_extentions_links = widgets.Textarea(value =
f"""https://github.com/hako-mikan/sd-webui-regional-prompter
https://github.com/OedoSoldier/enhanced-img2img
https://github.com/wcde/custom-hires-fix-for-automatic1111""", description='ссылки на репозитории дополнений (удали ссылки, если не нужны):', placeholder='только git-ссылки на репозитории дополнений').add_class("own_extentions_links")
def extentions_install():
  try:
    ownextentionslinks = own_extentions_links.value.splitlines()
    ownextentionslinks = [link for link in ownextentionslinks if not link.isspace()]
    if ownextentionslinks:
      def ext_sdextention_inst(ownextentionslink):
        subprocess.run(["git", "clone", ownextentionslink], cwd="/content/stable-diffusion-webui/extensions")
      with ThreadPoolExecutor(max_workers=len(ownextentionslinks)) as executor:
        futures = [executor.submit(ext_sdextention_inst, ownextentionslink) for ownextentionslink in ownextentionslinks]
        for future in as_completed(futures):
            future.result()
    if os.path.exists("/content/stable-diffusion-webui/extensions/custom-hires-fix-for-automatic1111/") and checkpoint_value == "previous":
      subprocess.run(["git", "checkout", "cd2f0689af06266fb7b92f362bbc5a544e750db4"], cwd="/content/stable-diffusion-webui/extensions/custom-hires-fix-for-automatic1111/")
      clear()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции extentions_install и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции extentions_install и получил "+str(e)
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)
  if extentions.value == False:
    None

# дополнения для миксов
def extentions_mix_install():
  try:
    if extentions_mix.value == True:
      mixextentions = [
      f"git clone https://github.com/hako-mikan/sd-webui-supermerger /content/stable-diffusion-webui/extensions/sd-webui-supermerger",
      f"git clone https://github.com/arenatemp/stable-diffusion-webui-model-toolkit /content/stable-diffusion-webui/extensions/stable-diffusion-webui-model-toolkit",
      f"git clone https://github.com/bbc-mc/sdweb-merge-block-weighted-gui /content/stable-diffusion-webui/extensions/sdweb-merge-block-weighted-gui",
      f"git clone https://github.com/bbc-mc/sdweb-merge-board /content/stable-diffusion-webui/extensions/sdweb-merge-board",
      ]
      def mixextention_inst(mixextention):
        !$mixextention
      with ThreadPoolExecutor(max_workers=len(mixextentions)) as executor:
        futures = [executor.submit(mixextention_inst, mixextention) for mixextention in mixextentions]
        for future in as_completed(futures):
            future.result()
    clear()
  except Exception as e:
    try:
      e = "в функции extentions_mix_install и получил "+str(e)
      error_message = str(cgitb.text(sys.exc_info()))
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      e = "в функции extentions_mix_install и получил "+str(e)
      import traceback
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)
  if extentions_mix.value == False:
    None

#скачивание интеррогейтов и языковых моделей для составления промпта (быстрее чем через вебуи)
def misc_models():
  try:
    miscmodels = [
    f"wget -q {gh}artists.txt -P /content/stable-diffusion-webui/interrogate",
    f"wget -q {gh}flavors.txt -P /content/stable-diffusion-webui/interrogate",
    f"wget -q {gh}mediums.txt -P /content/stable-diffusion-webui/interrogate",
    f"wget -q {gh}movements.txt -P /content/stable-diffusion-webui/interrogate",
    ]
    def misc_models_inst(miscmodel):
      !$miscmodel
    with ThreadPoolExecutor(max_workers=len(miscmodels)) as executor:
      futures = [executor.submit(misc_models_inst, miscmodel) for miscmodel in miscmodels]
      for future in as_completed(futures):
          future.result()
    gdshare_linker("interrogate/model-resnet_custom_v3.pt", "/content/stable-diffusion-webui/models/torch_deepdanbooru")
    gdshare_linker("interrogate/model_base_caption_capfilt_large.pth", "/content/stable-diffusion-webui/models/BLIP")
    clear()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции misc_models и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции misc_models и получил "+str(e)
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)

# некоторые модели апскелеров
def upscalers():
  try:
    modelsfolder = "/content/stable-diffusion-webui/models/"
    !rm -rf $modelsfolder'ESRGAN' $modelsfolder'LDSR' $modelsfolder'RealESRGAN' > /dev/null 2>&1
    gdshare_linker("ESRGAN", modelsfolder)
    gdshare_linker("RealESRGAN", modelsfolder)
    gdshare_linker("LDSR", modelsfolder)
    clear()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции upscalers и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции upscalers и получил "+str(e)
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)

def emdeddings_for_negative():
  try:
    webuifolder = "/content/stable-diffusion-webui/"
    for filename in os.listdir("/content/drive/MyDrive/colab_files/Lora"):
      gdshare_linker(f"Lora/{filename}", f"{webuifolder}models/Lora")
    for filename in os.listdir("/content/drive/MyDrive/colab_files/embeddings"):
      gdshare_linker(f"embeddings/{filename}", f"{webuifolder}embeddings")
    clear()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции emdeddings_for_negative и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции emdeddings_for_negative и получил "+str(e)
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)

# deepdanbooru и автозавершение тегов
def deepdanbooru():
  try:
    if booru_value == True:
      subprocess.run(["git", "clone", "--depth", "1", "https://github.com/DominikDoom/a1111-sd-webui-tagcomplete"], cwd="/content/stable-diffusion-webui/extensions")
      os.makedirs('/content/stable-diffusion-webui/tmp', exist_ok=True)
      tagacp = "/content/stable-diffusion-webui/tmp/tagAutocompletePath.txt"
      !touch $tagacp
      with open(tagacp, 'a') as ta: ta.write("extensions/a1111-sd-webui-tagcomplete/tags")
      !rm -rf /content/stable-diffusion-webui/models/deepbooru > /dev/null 2>&1
      gdshare_linker("deepbooru", "/content/stable-diffusion-webui/models/")
    if booru_value == False:
      None
  except Exception as e:
    try:
      e = "в функции deepdanbooru и получил "+str(e)
      error_message = str(cgitb.text(sys.exc_info()))
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      e = "в функции deepdanbooru и получил "+str(e)
      import traceback
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)
  clear()

# скачивание VAE
def VAE():
  try:
    models_path = "/content/stable-diffusion-webui/models/"
    for filename in os.listdir("/content/drive/MyDrive/colab_files/VAE"):
      gdshare_linker(f"VAE/{filename}", models_path)
    clear()
  except Exception as e:
    try:
      e = "в функции VAE и получил "+str(e)
      error_message = str(cgitb.text(sys.exc_info()))
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      e = "в функции VAE и получил "+str(e)
      import traceback
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)

# дефолтные конфиги
def configs():
  try:
    ng() #нгрок
    defconfigs = [
        f"{dl_gh}localization.json -O /content/stable-diffusion-webui/localizations/localization.json",
        (f"{dl} https://raw.githubusercontent.com/PR0LAPSE/StableDiffusionWebUIColab/88e98a11add26d13ecad35b171e03dbe6d3e5e68/src/config.json -O /content/stable-diffusion-webui/config.json" if checkpoint_value != "newest" else f"{dl_gh}config.json -O /content/stable-diffusion-webui/config.json"),
        (f"{dl} https://raw.githubusercontent.com/PR0LAPSE/StableDiffusionWebUIColab/88e98a11add26d13ecad35b171e03dbe6d3e5e68/src/ui-config.json -O /content/stable-diffusion-webui/ui-config.json" if checkpoint_value != "newest" else f"{dl_gh}ui-config.json -O /content/stable-diffusion-webui/ui-config.json")
    ]
    def defaultconf(defconfig):
        !$defconfig
    with ThreadPoolExecutor(max_workers=len(defconfigs)) as executor:
        futures = [executor.submit(defaultconf, defconfig) for defconfig in defconfigs]
        for future in as_completed(futures):
            future.result()
    # сохранение картинок на гулодиск
    if remaining_space(drive_service) > 69:
      replacer('/content/stable-diffusion-webui/config.json', '/content/output/', '/content/drive/MyDrive/SD/')
      replacer('/content/stable-diffusion-webui/config.json', '"control_net_allow_script_control": true', '"control_net_allow_script_control": true,\n    "def_batch_dir": "/content/drive/MyDrive/SD/ip2p-batch",\n    "outdir_ip2p_samples": "/content/drive/MyDrive/SD/ip2p-images"')
      if os.path.exists("/content/drive/MyDrive/SD/styles.csv"):
        !rm /content/stable-diffusion-webui/styles.csv /content/stable-diffusion-webui/styles.csv.bak
        !ln -s /content/drive/MyDrive/SD/styles.csv /content/stable-diffusion-webui/styles.csv
        !ln -s /content/drive/MyDrive/SD/styles.csv /content/stable-diffusion-webui/styles.csv.bak
        !chmod -c 777 /content/stable-diffusion-webui/styles.csv
        !chmod -c 777 /content/stable-diffusion-webui/styles.csv.bak
      else:
        !rm /content/stable-diffusion-webui/styles.csv /content/stable-diffusion-webui/styles.bak
        !$dl_gh'styles.csv' -O /content/drive/MyDrive/SD/styles.csv
        !ln -s /content/drive/MyDrive/SD/styles.csv /content/stable-diffusion-webui/styles.csv
        !ln -s /content/drive/MyDrive/SD/styles.csv /content/stable-diffusion-webui/styles.csv.bak
        !chmod -c 777 /content/stable-diffusion-webui/styles.csv
        !chmod -c 777 /content/stable-diffusion-webui/styles.csv.bak
    clear()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции configs и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции configs и получил "+str(e)
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)

# англоязычный интерфейс
def eng_lang():
  try:
    if eng.value == True:
      os.remove('/content/stable-diffusion-webui/localizations/localization.json')
      replacer('/content/stable-diffusion-webui/config.json', '"localization": "localization",', '"localization": "",')
    if eng.value == False:
      None
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции eng_lang и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      e = "в функции eng_lang и получил "+str(e)
      import traceback
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)

# кастомизация
def customisation():
  try:
    !rm /usr/local/lib/python3.10/dist-packages/gradio/strings.py
    !$dl_gh'gradio_strings.py' -O /usr/local/lib/python3.10/dist-packages/gradio/strings.py > /dev/null 2>&1
    !$dl_gh'notification.mp3' -O /content/stable-diffusion-webui/notification.mp3 > /dev/null 2>&1
    !rm /content/stable-diffusion-webui/html/footer.html /root/.ngrok2/ngrok.yml > /dev/null 2>&1
    if checkpoint_value == "previous":
      !$dl https://raw.githubusercontent.com/PR0LAPSE/StableDiffusionWebUIColab/88e98a11add26d13ecad35b171e03dbe6d3e5e68/src/footer.html -O /content/stable-diffusion-webui/html/footer.html > /dev/null 2>&1
    else:
      !$dl_gh'footer.html' -O /content/stable-diffusion-webui/html/footer.html > /dev/null 2>&1
    !$dl_gh'licenses.html' -O /content/stable-diffusion-webui/html/licenses.html > /dev/null 2>&1
    os.environ["DISPLAY"] = "ПРОЛАПС"
    without_wm = True
    if without_wm:
      replacer("/content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/scripts/gradio/inpainting.py", """def put_watermark(img, wm_encoder=None):\n    if wm_encoder is not None:\n        img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)\n        img = wm_encoder.encode(img, 'dwtDct')\n        img = Image.fromarray(img[:, :, ::-1])\n    return img""", "def put_watermark(img, wm_encoder=None):\n    return img")
      replacer("/content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/scripts/streamlit/inpainting.py", """def put_watermark(img, wm_encoder=None):\n    if wm_encoder is not None:\n        img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)\n        img = wm_encoder.encode(img, 'dwtDct')\n        img = Image.fromarray(img[:, :, ::-1])\n    return img""", "def put_watermark(img, wm_encoder=None):\n    return img")
      replacer("/content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/scripts/txt2img.py", """def put_watermark(img, wm_encoder=None):\n    if wm_encoder is not None:\n        img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)\n        img = wm_encoder.encode(img, 'dwtDct')\n        img = Image.fromarray(img[:, :, ::-1])\n    return img""", "def put_watermark(img, wm_encoder=None):\n    return img")
    replacer("/content/stable-diffusion-webui/modules/ui.py", "print('ngrok authtoken detected, trying to connect...')", "")
    replacer("/content/stable-diffusion-webui/modules/sd_models.py", 'f"Checkpoint {model_checkpoint} not found; loading fallback {checkpoint_info.title}", file=sys.stderr', '"загрузка модели..."')
    replacer("/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", '                strings.en["RUNNING_LOCALLY_SEPARATED"].format(\n                    self.protocol, self.server_name, self.server_port\n                )', '                  ""')
    replacer("/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", 'strings.en["SHARE_LINK_DISPLAY"].format(self.share_url)', '"\033[01;38;05;112m⯈\033[0m Ссылка на Градио:", self.share_url')
    replacer("/content/stable-diffusion-webui/modules/ngrok.py", "ngrok connected to localhost:{port}! URL: {public_url}\\n'\n               'You can use this link after the launch is complete.", "\\033[01;38;05;112m⯈\\033[0m Ссылка на Нгрок: {public_url}")
    replacer("/content/stable-diffusion-webui/modules/ngrok.py", "f'Invalid ngrok authtoken, ngrok connection aborted.\\n'\n              f'Your token: {token}, get the right one on https://dashboard.ngrok.com/get-started/your-authtoken'", "'указан неверный токен или тебе выпал общий токен который использует другой анон, получи свежий: https://dashboard.ngrok.com/get-started/your-authtoken'")
    replacer('/content/stable-diffusion-webui/javascript/ui.js',"""function restart_reload() {\n    document.body.innerHTML = '<h1 style="font-family:monospace;margin-top:20%;color:lightgray;text-align:center;">Reloading...</h1>';\n\n    var requestPing = function() {\n        requestGet("./internal/ping", {}, function(data) {\n            location.reload();\n        }, function() {\n            setTimeout(requestPing, 500);\n        });\n    };\n\n    setTimeout(requestPing, 2000);\n\n    return [];\n}""", 'function restart_reload(){\n    document.body.innerHTML=\'<h1 style="font-family:monospace;margin-top:20%;color:red;text-align:center;">Чтобы выполнить перезагрузку, нужно остановить выполнение кода в ячейке колаба:<br>Ctrl+M+I или "Среда выполнения -> Остановить выполнение кода"<br>Затем нужно выполнить перезапуск через ячейку перезапуска!<br>Штатная функция перезапуска ВебУИ не работает через колаб!!!<br>Останавливать и перезапускать нужно каждый раз после добавления/изменения дополнений, списка экстрасетей, скриптов, и т.п.<br>Это необходимо из-за использования каждый раз разных веб-адресов вебуи, в отличии от локальной установки.</h1>\';\n    setTimeout(function(){location.reload()},40000)\n\n    return []\n}')
    replacer("/usr/local/lib/python3.10/dist-packages/pyngrok/conf.py", """DEFAULT_NGROK_CONFIG_PATH = os.path.join(os.path.expanduser("~"), ".ngrok2", "ngrok.yml")""", '''DEFAULT_NGROK_CONFIG_PATH = "/root/.config/ngrok/ngrok.yml"''')
    replacer("/content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py", """if verbose:\n        print(f"{model.__class__.__name__} has {total_params*1.e-6:.2f} M params.")""", "")
    replacer("/content/stable-diffusion-webui/extensions-builtin/LDSR/sd_hijack_ddpm_v1.py", """print(f"{self.__class__.__name__}: Running in {self.parameterization}-prediction mode")""", "")
    replacer("/content/stable-diffusion-webui/modules/models/diffusion/ddpm_edit.py", """print(f"{self.__class__.__name__}: Running in {self.parameterization}-prediction mode")""", "")
    replacer("/content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/models/diffusion/ddpm.py", """print(f"{self.__class__.__name__}: Running in {self.parameterization}-prediction mode")""", "")
    replacer("/content/stable-diffusion-webui/webui.py", """print(f"Startup time: {startup_timer.summary()}.")""", "")
    replacer("/content/stable-diffusion-webui/webui.py", """print(f"Startup time: {startup_timer.summary()}.")""", "")
    replacer("/content/stable-diffusion-webui/webui.py", """print(f'Interrupted with signal {sig} in {frame}')""", "")
    replacer("/content/stable-diffusion-webui/modules/sd_models.py", """print(f"Model loaded in {timer.summary()}.")""", "")
    replacer("/content/repositories/stable-diffusion-stability-ai/ldm/modules/diffusionmodules/model.py", """    print("No module 'xformers'. Proceeding without it.")""", "")
    replacer("/content/stable-diffusion-webui/modules/textual_inversion/textual_inversion.py", """print(f"Textual inversion embeddings loaded({len(self.word_embeddings)}): {', '.join(self.word_embeddings.keys())}")""", "")
    replacer("/content/stable-diffusion-webui/modules/textual_inversion/textual_inversion.py", """                print(f"Textual inversion embeddings skipped({len(self.skipped_embeddings)}): {', '.join(self.skipped_embeddings.keys())}")""", "                None")
    replacer("/content/stable-diffusion-webui/extensions/sd-webui-additional-networks/scripts/model_util.py", """print("[AddNet] Updating model hashes...")""", "")
    replacer("/content/stable-diffusion-webui/extensions/sd-webui-additional-networks/scripts/model_util.py", """thread_count = max(1, int(shared.opts.data.get("additional_networks_hash_thread_count", 1)))""", """thread_count = 4""")
    replacer("/content/stable-diffusion-webui/modules/ui_extensions.py", """    print(\n        f"preload_extensions_git_metadata for "\n        f"{len(extensions.extensions)} extensions took "\n        f"{time.time() - t0:.2f}s"\n    )""", "")
    replacer("/content/stable-diffusion-webui/extensions/a1111-sd-webui-lycoris/scripts/lycoris_script.py", """        logger.warning('Triggered lyco-patch-lora, will take lora_dir and <lora> format.')""", "")
    replacer("/content/stable-diffusion-webui/extensions/a1111-sd-webui-lycoris/scripts/lycoris_script.py", """        logger.debug("Set lyco logger level to DEBUG")""", "")
    replacer("/content/stable-diffusion-webui/extensions/adetailer/scripts/!adetailer.py", """print(\n    f"[-] ADetailer initialized. version: {__version__}, num models: {len(model_mapping)}"\n)""", "")
    replacer("/content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/annotator_path.py", """print(f'ControlNet preprocessor location: {models_path}')""", "")
    replacer("/content/stable-diffusion-webui/extensions/sd-webui-controlnet/preload.py", """        default="INFO",""", """        default="ERROR",""")
    replacer("/content/stable-diffusion-webui/extensions/sd-webui-controlnet/scripts/logging.py", """loglevel_string = getattr(shared.cmd_opts, "controlnet_loglevel", "INFO")""", """loglevel_string = getattr(shared.cmd_opts, "controlnet_loglevel", "ERROR")""")
    replacer("/content/stable-diffusion-webui/extensions/a1111-sd-webui-lycoris/scripts/lycoris_script.py", """        logger.warning('Triggered lyco-patch-lora, will take lora_dir and <lora> format.')""", "")
    replacer("/content/stable-diffusion-webui/extensions/sd-webui-regional-prompter/scripts/rp.py", """        print("Init / preset error.")""", "")
    replacer("/content/stable-diffusion-webui/extensions/sd-webui-regional-prompter/scripts/rp.py", """        print("Presets could not be loaded.") """, "")
    replacer("/content/stable-diffusion-webui/modules/ui_extensions.py", """    print(\n        f"preload_extensions_git_metadata for "\n        f"{len(extensions.extensions)} extensions took "\n        f"{time.time() - t0:.2f}s"\n    )""", "")
    replacer("/content/stable-diffusion-webui/extensions/sd-webui-roop-nsfw/scripts/roop_logging.py", """loglevel_string = getattr(shared.cmd_opts, "roop_loglevel", "INFO")""", """loglevel_string = getattr(shared.cmd_opts, "roop_loglevel", "WARNING")""")
    replacer("/content/stable-diffusion-webui/extensions/sd-webui-roop-nsfw/scripts/roop_logging.py", """loglevel = getattr(logging, loglevel_string.upper(), "info")""", """loglevel = getattr(logging, loglevel_string.upper(), "warning")""")
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции customisation и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции customisation и получил "+str(e)
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)
  #clear()

# удалить пустые папки .ipynb_checkpoints и мусор
def cleanup():
  try:
    shutil.rmtree("/content/sample_data")
  except:
    None
  sdwu = "/content/stable-diffusion-webui/"
  junk = ["webui.sh", "webui.bat", "webui-user.sh", "webui-user.bat", "webui-macos-env.sh", "screenshot.png", "README.md", "LICENSE.txt", "CODEOWNERS",]
  dirs_to_remove = glob.glob("/content/.ipynb_checkpoints", recursive=True)
  [os.remove(os.path.join(sdwu, file)) for file in junk if os.path.exists(os.path.join(sdwu, file))]
  for dir in dirs_to_remove:
    path = os.path.join(sdwu, dir)
    try:
      shutil.rmtree(path)
    except:
      None
  if os.path.exists("/root/.ngrok2/ngrok.yml"):
    try:
      shutil.move("/root/.ngrok2/ngrok.yml", "/root/.config/ngrok/ngrok.yml")
    except:
      None
  for root, dirs, files in os.walk("/content/stable-diffusion-webui/"):
    for file in files:
      if file.endswith('here.txt'):
        os.remove(os.path.join(root, file))

def complite():
  try:
    clear()
    import psutil
    from subprocess import getoutput
    from IPython.display import HTML
    total_ram = round(psutil.virtual_memory().total / (1024**3), 2)
    free_ram = round(psutil.virtual_memory().available / (1024**3), 2)
    allocated_ram = round(total_ram - free_ram, 2)
    percent = round((free_ram / total_ram) * 100, 2)
    display(HTML("""<style>.setup_complite{color:var(--ansi-green);font-weight:700}hr.hr_setup_comlite{width:88%;position:relative;right:50%;border:none;border-top:2px var(--ansi-green) solid}.memory{color:var(--ansi-blue);font-weight:bold}.mem_allocate,.mem_all{color:var(--ansi-green)}.percent{color:var(--colab-primary-text-color)}.little_memory{display:flex;flex-wrap:wrap;width:50%;color:var(--ansi-red);font-weight:bold;font-family:sans-serif!important}.cell_fix{font-family:Consolas,monospace!important;color:var(--ansi-blue)}.cell_fix:before{content:"«";font-family:sans-serif!important;color:var(--ansi-blue)}.cell_fix:after{content:"»";font-family:sans-serif!important;color:var(--ansi-blue)}</style>"""))
    display(HTML(f'<p class="setup_complite">Установка завершена<hr class="hr_setup_comlite"/><span class="memory">Доступно ОЗУ: <span class="mem_allocate">{free_ram} Гб</span> из <span class="mem_all">{total_ram} Гб</span> <span class="percent">({percent}% свободно)</span></span></p>'))
    if int(free_ram) <= 7:
      display(HTML('<p class="little_memory">Осталось слишком мало памяти!<br/>Рекомендуется остановить ячейку и выполнить сброс памяти через ячейку <span class="cell_fix">фикс утечки памяти</span>, что находится выше</p>'))
    nsmi = getoutput('nvidia-smi')
    if "nvidia-smi: not found" in nsmi or "NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running." in nsmi:
      display(HTML(f'{gh}warn'))
    else:
      try:
        card = getoutput('nvidia-smi --query-gpu=gpu_name --format=csv').split('\n', 1)[1]
        vram = getoutput('nvidia-smi --query-gpu=memory.total --format=csv').split('\n', 1)[1]
        print("Видеокарта: \033[01;38;05;46mNvidia " + card + "\033[0m |" + "\033[01;38;05;188m Объём видеопамяти: \033[01;38;05;46m" + vram + "\033[0m ")
      except:
        print("nvidia-smi не ответил...")
    aloud("finish.wav")
    if starton_value == False:
      if opt_value == 'SDP' or opt_value == 'ToMe':
        xf = '--opt-sdp-attention'
      if opt_value == 'xformers':
        xf = '--xformers'
      if opt_value == 'nothing':
        xf = '--opt-split-attention'
      if no_half_vae_value == True:
        nhv = '--no-half-vae'
      if no_half_vae_value == False:
        nhv = ''
      gq = '--disable-console-progressbars --no-hashing'
      ckpt_dir = "/content/stable-diffusion-webui/models/Stable-diffusion"
      if booru_value == True:
        deepdanbooru_arg = "--deepdanbooru"
      if booru_value == False:
        deepdanbooru_arg = ""
      default_args = f"--disable-safe-unpickle --enable-insecure-extension-access --share --listen {deepdanbooru_arg} --theme dark --ckpt-dir {ckpt_dir}"
      ngrok_share = cfg+'cfg'
      freetoken = open(ngrok_share).read().split('\n')
      sharetoken = random.choice(list(freetoken))
      token = ngrok_token
      if not ngrok_token_value or len(ngrok_token_value) < 48 or len(ngrok_token_value) > 51:
        ngrok = f'--ngrok {sharetoken} --ngrok-region eu'
      else:
        ngrok = f'--ngrok {ngrok_token_value} --ngrok-region eu'
      cmdargs = f'{default_args} {gq} {xf} {nhv} {ngrok}'
      # сохранение переменных
      args_data = (xf, nhv, gq, deepdanbooru_arg, default_args)
      with open(f'{cfg}webui_args', 'wb') as webui_args: pickle.dump(args_data, webui_args)
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции complite и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции complite и получил "+str(e)
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)

def run_webui():
  if starton_value == True:
    try:
      !sleep 3
      aloud("start.mp3")
      !sleep 3
      #Установка токена ngrok
      ngrok_share = cfg+'cfg'
      freetoken = open(ngrok_share).read().split('\n')
      sharetoken = random.choice(list(freetoken))
      token = ngrok_token
      if not ngrok_token_value or len(ngrok_token_value) < 48 or len(ngrok_token_value) > 51:
        ngrok = f'--ngrok {sharetoken} --ngrok-region eu'
      else:
        ngrok = f'--ngrok {ngrok_token_value} --ngrok-region eu'
      if opt_value == 'SDP' or opt_value == 'ToMe':
        xf = '--opt-sdp-attention'
      if opt_value == 'xformers':
        xf = '--xformers'
      if opt_value == 'nothing':
        xf = '--opt-split-attention'
      if no_half_vae_value == True:
        nhv = '--no-half-vae'
      if no_half_vae_value == False:
        nhv = ''
      gq = '--disable-console-progressbars --no-hashing'
      ckpt_dir = "/content/stable-diffusion-webui/models/Stable-diffusion"
      if booru_value == True:
        deepdanbooru_arg = "--deepdanbooru"
      if booru_value == False:
        deepdanbooru_arg = ""
      default_args = f"--disable-safe-unpickle --enable-insecure-extension-access --share --listen {deepdanbooru_arg} --theme dark --ckpt-dir {ckpt_dir}"
      cmdargs = f'{default_args} {gq} {xf} {nhv} {ngrok}'
      # сохранение переменных
      args_data = (xf, nhv, gq, deepdanbooru_arg, default_args)
      with open(f'{cfg}webui_args', 'wb') as webui_args: pickle.dump(args_data, webui_args)
      # запуск
      print("\033[1;36mВнимание! При проблемах с интерфейсом, используй другую ссылку, или перезагружай страницу, или выполни перезапуск.\033[0m \n")
      print("\033[1;34mПереходить по ссылкам можно после полной прогрузки, ожидай сообщения.\nСылки могут быть открыты автоматически, если в браузере выданы разрешения на переадресации и открытие окон для этого сайта.\033[0m")
      flara()
      print("")
      #Собственно запуск
      auto_opener()
      # в последней версии автоматик добавил аргументы: --no-hashing, можно использовать вместе с --lowram
      startwu = f"python /content/stable-diffusion-webui/webui.py {cmdargs}"
      !rm /root/.ngrok2/ngrok.yml > /dev/null 2>&1
      !$startwu
    except Exception as e:
      try:
        error_message = str(cgitb.text(sys.exc_info()))
        e = "в функции run_webui и получил "+str(e)
        ErrorReport(e, error_message)
        print("Ошибка: ", e)
      except Exception as e:
        e = "в функции run_webui и получил "+str(e)
        import traceback
        error_message = str(traceback.format_exc())
        ErrorReport(e, error_message)
        print("Ошибка: ", error_message)
    if os.path.exists("/content/drive/MyDrive/SD/styles.csv"):
      !cp -f /content/stable-diffusion-webui/styles.csv /content/drive/MyDrive/SD/styles.csv
    display(HTML('/content/.config/end'))
  else:
    print("\033[01;38;05;211mтобой был отменен автозапуск после установки:\n\033[01;38;05;211mчтобы выполнить запуск вебуи, нужно запустить ячейку «запуск/перезапуск webui».")

# еще один костыль: доустановка зависимостей после установки дополнений
def misc_dep_install():
  try:
    print("завершение установки...")
    os.chdir('/content/stable-diffusion-webui')
    loader(inprogress)
    subprocess.run(["python", "launch.py ", "--share ", "--xformers", "--ngrok", "test", "--no-download-sd-model", "--exit"], cwd="/content/stable-diffusion-webui")
    if checkpoint_value == "previous":
      subprocess.run(["pip", "install", "gradio==3.33.0"])
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции misc_dep_install и получил "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      e = "в функции misc_dep_install и получил "+str(e)
      error_message = str(traceback.format_exc())
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)
def depinst():
  if checkpoint_value == "newest":
    depinst_newest()
  elif checkpoint_value == "previous":
    depinst_previous()
  else:
    None

# ver 0.6.4 (08.09.2023 - фикс дублирования папки) Примонтировать модели, эмбендинги, лоры, и прочее с гугл диска с костылём для симликнутых папок
# симлинки с гуглодиска
def ultimate_gdrive_symlink():
  BlackLivesMatter = "st"+"ab"+"le-di"+"ffu"+"si"+"on"+"-w"+"e"+"bui"
  LGBT = "Sta"+"bl"+"e-d"+"if"+"fus"+"ion"
  excluded_dirs = ['txt2img-grids', 'img2img-grids', 'img2img-images', 'txt2img-images', 'extras-images', 'ComfyUI', 'fine']
  file_extensions = ('.pt', '.safetensors', '.ckpt', '.bin', '.pth')
  def has_file_with_extension(directory):
      for root, _, files in os.walk(directory):
          for file in files:
              if file.endswith(file_extensions):
                  return True
      return False
  def is_directory_empty(directory):
      cmd = ['find', directory, '-mindepth', '1', '-maxdepth', '1', '-print', '-quit']
      result = subprocess.run(cmd, shell=False, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
      return result.returncode != 0
  def create_symlink(source, dest_dir, name):
      dest_path = os.path.join(dest_dir, name)
      try:
          os.system(f'ln -s {source} {dest_path}')
          if is_directory_empty(dest_dir):
              os.symlink(source, dest_path)
      except FileExistsError:
          pass
      except OSError:
          pass
  def process_dir(src_dir, dest_dir):
      os.makedirs(dest_dir, exist_ok=True)
      for entry in os.scandir(src_dir):
          if entry.name not in excluded_dirs:
              if entry.is_dir():
                  if has_file_with_extension(entry.path):
                      new_dest_dir = os.path.join(dest_dir, entry.name)
                      os.makedirs(new_dest_dir, exist_ok=True)
                      for file in os.listdir(entry.path):
                          if file.endswith(file_extensions):
                              create_symlink(os.path.join(entry.path, file), new_dest_dir, file)
                  else:
                      process_dir(entry.path, os.path.join(dest_dir, entry.name))
              elif entry.is_file() and entry.name.endswith(file_extensions):
                  create_symlink(entry.path, dest_dir, entry.name)

  source_dir = "/content/drive/MyDrive/SD/"
  destination_dir = f"/content/{BlackLivesMatter}/"
  if not source_dir.startswith("/content/drive/MyDrive/"):
      source_dir = os.path.join("/content/drive/MyDrive/", source_dir)
  os.makedirs(destination_dir, exist_ok=True)
  if os.path.isdir(source_dir) and not is_directory_empty(source_dir):
      with concurrent.futures.ThreadPoolExecutor(max_workers=len(os.listdir(source_dir))) as executor:
          for entry in os.scandir(source_dir):
              if entry.name not in excluded_dirs:
                  if entry.is_dir() or (entry.is_file() and entry.name.endswith(file_extensions)):
                      executor.submit(process_dir, entry.path, os.path.join(destination_dir, os.path.relpath(entry.path, source_dir)))

  if not (os.path.isdir(source_dir) or os.path.exists(source_dir)):
    print(f'Еггог: {source_dir} путь неверный')
    #fallback, без папок, длинный вариант, который работает
  from subprocess import getstatusoutput
  status, output = getstatusoutput(f'find /content/{BlackLivesMatter}/models/Lora/ -type l -print -quit')
  if status != 0:
    print('ошибка:\n', output)
  else:
    if not output:
      os.makedirs(f'/content/{BlackLivesMatter}/models/Lora', exist_ok=True)
      files = glob.glob('/content/drive/MyDrive/SD/models/Lora/*')
      for file in files:
        subprocess.run(['ln', '-s', file, f'/content/{BlackLivesMatter}/models/Lora/'])
  status, output = getstatusoutput(f'find /content/{BlackLivesMatter}/models/{LGBT}/ -type l')
  if status != 0:
    print('ошибка:\n', output)
  else:
    if not output:
      files = glob.glob(f'/content/drive/MyDrive/SD/models/{LGBT}/*')
      for file in files:
        subprocess.run(['ln', '-s', file, f'/content/{BlackLivesMatter}/models/{LGBT}/'])
  status, output = getstatusoutput(f'find /content/{BlackLivesMatter}/models/hypernetworks/ -type l')
  if status != 0:
    print('ошибка:\n', output)
  else:
    if not output:
      files = glob.glob('/content/drive/MyDrive/SD/models/hypernetworks/*')
      for file in files:
        subprocess.run(['ln', '-s', file, f'/content/{BlackLivesMatter}/models/hypernetworks/'])
  status, output = getstatusoutput(f'find /content/{BlackLivesMatter}/embeddings/ -type l')
  if status != 0:
    print('ошибка:\n', output)
  else:
    if not output:
      files = glob.glob('/content/drive/MyDrive/SD/embeddings/*')
      for file in files:
        subprocess.run(['ln', '-s', file, f'/content/{BlackLivesMatter}/embeddings/'])
  # удаление "*"
  for root, dirs, files in os.walk(f'/content/{BlackLivesMatter}/models'):
    for file in files:
      if file == '*':
        os.remove(os.path.join(root, file))
  if os.path.exists(f"/content/{BlackLivesMatter}/embeddings/*"):
    os.remove(f"/content/{BlackLivesMatter}/embeddings/*")

# фикс жора ОЗУ
def ram_liberator_old():
    debs = [
        "wget -q http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb",
        "wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb",
        "wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb",
        "wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb",
        "apt install -qq libunwind8-dev",
    ]
    def colab_downgrade(deb):
        subprocess.getoutput(deb)
    with ThreadPoolExecutor(max_workers=len(debs)) as executor:
        futures = [executor.submit(colab_downgrade, deb) for deb in debs]
        for future in futures:
            future.result()
    subprocess.getoutput("dpkg -i *.deb && rm  *.deb")
def ram_liberator_jemalloc():
  !apt -y update -qq > /dev/null 2>&1
  !sudo apt-get install libjemalloc-dev > /dev/null 2>&1
  !sudo rm /usr/local/lib/libtbbmalloc.so.2 > /dev/null 2>&1
  !sudo ln -s /usr/lib/x86_64-linux-gnu/libtbbmalloc.so.2 /usr/local/lib/libtbbmalloc.so.2 > /dev/null 2>&1
  !sudo rm /usr/local/lib/libtbbmalloc_proxy.so.2 > /dev/null 2>&1
  !sudo ln -s /usr/lib/x86_64-linux-gnu/libtbbmalloc_proxy.so.2 /usr/local/lib/libtbbmalloc_proxy.so.2 > /dev/null 2>&1
  !sudo rm /usr/local/lib/libtbb.so.12 > /dev/null 2>&1
  !sudo ln -s /usr/lib/x86_64-linux-gnu/libtbb.so.12 /usr/local/lib/libtbb.so.12 > /dev/null 2>&1
  !sudo rm /usr/local/lib/libtbbbind_2_5.so.3 > /dev/null 2>&1
  !sudo ln -s /usr/lib/x86_64-linux-gnu/libtbbbind_2_5.so.3 /usr/local/lib/libtbbbind_2_5.so.3 > /dev/null 2>&1
  !sudo rm /usr/local/lib/libtbbbind.so.3 > /dev/null 2>&1
  !sudo ln -s /usr/lib/x86_64-linux-gnu/libtbbbind.so.3 /usr/local/lib/libtbbbind.so.3 > /dev/null 2>&1
  !sudo rm /usr/local/lib/libtbbbind_2_0.so.3 > /dev/null 2>&1
  !sudo ln -s /usr/lib/x86_64-linux-gnu/libtbbbind_2_0.so.3 /usr/local/lib/libtbbbind_2_0.so.3 > /dev/null 2>&1

def ram_liberator():
  !$wget $gh'libtcmalloc_minimal.so.4' -O '/lib/libtcmalloc_minimal.so.4' > /dev/null 2>&1

# все действия после нажатия на кнопку "продолжить"
# ver 19.02.2023
def parallels_tasks():
    tasks = [
        extentions_builtin,
        extentions_install,
        extentions_mix_install,
        upscalers,
        emdeddings_for_negative,
        VAE,
        deepdanbooru,
        instruct_pix2pix,
        ControlNet_install,
        ultimate_gdrive_symlink,
        misc_models,
    ]
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor: #временно 16 вместо (max_workers=len(tasks)) чтобы не задыхалось
        # шлем каждый task в отдельный executor
        futures = [executor.submit(run_task, task) for task in tasks]
        for future in concurrent.futures.as_completed(futures):
            # ждем завершения, выплевываем ошибки
            try:
                future.result()
            except Exception as e:
              try:
                error_message = str(cgitb.text(sys.exc_info()))
                e = "в функции parallels_tasks и получил "+str(e)
                ErrorReport(e, error_message)
                print("Ошибка: ", e)
              except Exception as e:
                import traceback
                e = "в функции parallels_tasks и получил "+str(e)
                error_message = str(traceback.format_exc())
                ErrorReport(e, error_message)
                print("Ошибка: ", error_message)

def run_task(task):
    # новый ThreadPoolExecutor для каждого task
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # запускаем каждый task в новом executor
        executor.submit(task)

#  ram_liberator()
#  ultimate_gdrive_symlink()
def apply_button_click(b): # выполнение последрвательно всего выше (паузы  внутри самих def)
  try:
    clear()
    print("жди, установка уже началась...")
    run_nb("store_values")
    depinst()
    A1111_webui_install()
    parallels_tasks()
    configs()
    eng_lang()
    misc_dep_install()
    customisation()
    cleanup()
    complite()
    run_webui()
  except Exception as e:
    try:
      error_message = str(cgitb.text(sys.exc_info()))
      e = "в функции apply_button_click(b) "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", e)
    except Exception as e:
      import traceback
      error_message = str(traceback.format_exc())
      e = "в функции apply_button_click(b) "+str(e)
      ErrorReport(e, error_message)
      print("Ошибка: ", error_message)
